<a href="https://colab.research.google.com/github/jagathcharan/VQA_CNN_AND_LSTM/blob/main/VQA_CNN_AND_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data loading

In [ ]:
import numpy as np
import os

from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image

INPUT_DIR = '../data'

class VQADataset(Dataset):

    def __init__(self, input_dir, input_file, max_qu_len = 30, transform = None):

        self.input_data = np.load(os.path.join(input_dir, input_file), allow_pickle=True)
        self.qu_vocab = Vocab(input_dir+'/question_vocabs.txt')
        self.ans_vocab = Vocab(input_dir+'/annotation_vocabs.txt')
        self.max_qu_len = max_qu_len
        self.labeled = True if not "test" in input_file else False
        self.transform = transform

    def __getitem__(self, idx):

        path = self.input_data[idx]['img_path']
        img = np.array(Image.open(path).convert('RGB'))
        qu_id = self.input_data[idx]['qu_id']
        qu_tokens = self.input_data[idx]['qu_tokens']
        qu2idx = np.array([self.qu_vocab.word2idx('<pad>')] * self.max_qu_len)
        qu2idx[:len(qu_tokens)] = [self.qu_vocab.word2idx(token) for token in qu_tokens]
        sample = {'image': img, 'question': qu2idx, 'question_id': qu_id}

        if self.labeled:
            ans2idx = [self.ans_vocab.word2idx(ans) for ans in self.input_data[idx]['valid_ans']]
            ans2idx = np.random.choice(ans2idx)
            sample['answer'] = ans2idx

        if self.transform:
            sample['image'] = self.transform(sample['image'])

        return sample

    def __len__(self):

        return len(self.input_data)

def data_loader(input_dir, batch_size, max_qu_len, num_worker):

    transform = transforms.Compose([
        transforms.ToTensor(),  # convert to (C,H,W) and [0,1]
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # mean=0; std=1
    ])

    vqa_dataset = {
        'train': VQADataset(
            input_dir=input_dir,
            input_file='train.npy',
            max_qu_len=max_qu_len,
            transform=transform),
        'val': VQADataset(
            input_dir=input_dir,
            input_file='val.npy',
            max_qu_len=max_qu_len,
            transform=transform)
    }

    dataloader = {
        key: DataLoader(
            dataset=vqa_dataset[key],
            batch_size=batch_size,
            shuffle=True,
            num_workers=num_worker)
        for key in ['train', 'val']
    }

    return dataloader

class Vocab:

    def __init__(self, vocab_file):

        self.vocab = self.load_vocab(vocab_file)
        self.vocab2idx = {vocab: idx for idx, vocab in enumerate(self.vocab)}
        self.vocab_size = len(self.vocab)

    def load_vocab(self, vocab_file):

        with open(vocab_file) as f:
            vocab = [v.strip() for v in f]

        return vocab

    def word2idx(self, vocab):

        if vocab in self.vocab2idx:
            return self.vocab2idx[vocab]
        else:
            return self.vocab2idx['<unk>']

    def idx2word(self, idx):

        return self.vocab[idx]

##Training

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F

class ImgEncoder(nn.Module):
    def __init__(self, embed_dim):
        super(ImgEncoder, self).__init__()
        self.model = models.resnet50(pretrained=True)
        in_features = self.model.fc.in_features
        self.model.fc = nn.Identity()  # Remove the final classification layer
        self.fc = nn.Linear(in_features, embed_dim)

    def forward(self, image):
        with torch.no_grad():
            img_feature = self.model(image)  # (batch, feature_size)
        img_feature = self.fc(img_feature)
        l2_norm = F.normalize(img_feature, p=2, dim=1).detach()
        return l2_norm

class QuEncoder(nn.Module):
    def __init__(self, qu_vocab_size, word_embed, hidden_size, num_hidden, qu_feature_size):
        super(QuEncoder, self).__init__()
        self.word_embedding = nn.Embedding(qu_vocab_size, word_embed)
        self.tanh = nn.Tanh()
        self.lstm = nn.LSTM(word_embed, hidden_size, num_hidden)  # input_feature, hidden_feature, num_layer
        self.fc = nn.Linear(2*num_hidden*hidden_size, qu_feature_size)

    def forward(self, question):
        qu_embedding = self.word_embedding(question)                # (batchsize, qu_length=30, word_embed=300)
        qu_embedding = self.tanh(qu_embedding)
        qu_embedding = qu_embedding.transpose(0, 1)                 # (qu_length=30, batchsize, word_embed=300)
        _, (hidden, cell) = self.lstm(qu_embedding)                 # (num_layer=2, batchsize, hidden_size=1024)
        qu_feature = torch.cat((hidden, cell), dim=2)               # (num_layer=2, batchsize, 2*hidden_size=1024)
        qu_feature = qu_feature.transpose(0, 1)                     # (batchsize, num_layer=2, 2*hidden_size=1024)
        qu_feature = qu_feature.reshape(qu_feature.size()[0], -1)   # (batchsize, 2*num_layer*hidden_size=2048)
        qu_feature = self.tanh(qu_feature)
        qu_feature = self.fc(qu_feature)                            # (batchsize, qu_feature_size=1024)
        return qu_feature

class VQAModel(nn.Module):
    def __init__(self, feature_size, qu_vocab_size, ans_vocab_size, word_embed, hidden_size, num_hidden):
        super(VQAModel, self).__init__()
        self.img_encoder = ImgEncoder(feature_size)
        self.qu_encoder = QuEncoder(qu_vocab_size, word_embed, hidden_size, num_hidden, feature_size)
        self.dropout = nn.Dropout(0.5)
        self.tanh = nn.Tanh()
        self.fc1 = nn.Linear(feature_size, ans_vocab_size)
        self.fc2 = nn.Linear(ans_vocab_size, ans_vocab_size)

    def forward(self, image, question):
        img_feature = self.img_encoder(image)               # (batchsize, feature_size=1024)
        qst_feature = self.qu_encoder(question)
        combined_feature = img_feature * qst_feature
        combined_feature = self.dropout(combined_feature)
        combined_feature = self.tanh(combined_feature)
        combined_feature = self.fc1(combined_feature)       # (batchsize, ans_vocab_size=1000)
        combined_feature = self.dropout(combined_feature)
        combined_feature = self.tanh(combined_feature)
        logits = self.fc2(combined_feature)                 # (batchsize, ans_vocab_size=1000)
        return logits


##Testing

In [ ]:
import os
import time
import torch
import torch.nn as nn

from torch import optim
from model import VQAModel
from build_dataset import data_loader

DATA_DIR = '../data'
CKPT_DIR = '../ckpt'
LOG_DIR = '../log'

BATCH_SIZE = 150
MAX_QU_LEN = 30
NUM_WORKER = 8
FEATURE_SIZE, WORD_EMBED = 1024, 300
NUM_HIDDEN, HIDDEN_SIZE = 2, 512
LEARNING_RATE, STEP_SIZE, GAMMA = 0.001, 10, 0.1
EPOCH = 50

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

def train():

    dataloader = data_loader(input_dir=DATA_DIR, batch_size=BATCH_SIZE, max_qu_len=MAX_QU_LEN, num_worker=NUM_WORKER)
    qu_vocab_size = dataloader['train'].dataset.qu_vocab.vocab_size
    ans_vocab_size = dataloader["train"].dataset.ans_vocab.vocab_size

    model = VQAModel(feature_size=FEATURE_SIZE, qu_vocab_size=qu_vocab_size, ans_vocab_size=ans_vocab_size,
                     word_embed=WORD_EMBED, hidden_size=HIDDEN_SIZE, num_hidden=NUM_HIDDEN).to(device)

    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)
    criterion = nn.CrossEntropyLoss()

    print('>> start training')
    start_time = time.time()
    for epoch in range(EPOCH):
        epoch_loss = {key: 0 for key in ['train', 'val']}

        model.train()
        for idx, sample in enumerate(dataloader['train']):

            image = sample['image'].to(device=device)
            question = sample['question'].to(device=device)
            label = sample['answer'].to(device=device)
            # forward
            logits = model(image, question)
            loss = criterion(logits, label)
            epoch_loss['train'] += loss.item()
            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        for idx, sample in enumerate(dataloader['val']):

            image = sample['image'].to(device=device)
            question = sample['question'].to(device=device)
            label = sample['answer'].to(device=device)
            with torch.no_grad():
                logits = model(image, question)
                loss = criterion(logits, label)
            epoch_loss['val'] += loss.item()

        # statistic
        for phase in ['train', 'val']:
            epoch_loss[phase] /= len(dataloader[phase])
            with open(os.path.join(LOG_DIR, f'{phase}_log.txt'), 'a') as f:
                f.write(str(epoch+1) + '\t' + str(epoch_loss[phase]) + '\n')
        print('Epoch:{}/{} | Training Loss: {train:6f} | Validation Loss: {val:6f}'.format(epoch+1, EPOCH, **epoch_loss))

        scheduler.step()
        early_stop = early_stopping(model, epoch_loss['val'])
        if (epoch+1) % 5 == 0:
            torch.save(model.state_dict(), os.path.join(CKPT_DIR, f'model-epoch-{epoch+1}.pth'))
        if early_stop:
            print(f'>> Early stop at {epoch+1} epoch')
            break

    end_time = time.time()
    training_time = end_time - start_time
    print(f">> Finishing training | Training Time:{training_time//60:.0f}m:{training_time%60:.0f}s")

def early_stopping(model, epoch_loss, patience=7):

    early_stop = False
    if not bool(early_stopping.__dict__):
        early_stopping.best_loss = epoch_loss
        early_stopping.record_loss = epoch_loss
        early_stopping.counter = 0

    if epoch_loss < early_stopping.best_loss:
        early_stopping.best = epoch_loss
        torch.save(model.state_dict(), os.path.join(CKPT_DIR, 'best_model.pth'))

    if epoch_loss > early_stopping.record_loss:
        early_stopping.counter += 1
        if early_stopping.counter >= patience:
            early_stop = True
    else:
        early_stopping.counter = 0
        early_stopping.record_loss = epoch_loss

    return early_stop

if __name__ == '__main__':

    if not os.path.exists(LOG_DIR):
        os.makedirs(LOG_DIR)
    if not os.path.exists(CKPT_DIR):
        os.makedirs(CKPT_DIR)
    train()

In [ ]:
import os
import json
import torch

from torchvision import transforms
from torch.utils.data import DataLoader
from model import VQAModel
from build_dataset import VQADataset

device = torch.device('cuda')
data_dir = '../data'
ckpt_dir = '../ckpt/best_model.pth'
res_dir = '../Results'

BATCH_SIZE = 128
FEATURE_SIZE, WORD_EMBED = 1024, 300
MAX_QU_LEN, NUM_HIDDEN, HIDDEN_SIZE = 30, 2, 512

def test(input_dir, data_type, batch_size, num_worker):

    """
    results = [result]
    result{ "question_id": int,
            "answer": str }......
    """
    transform = transforms.Compose([
        transforms.ToTensor(),  # convert to (C,H,W) and [0,1]
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # mean=0; std=1
    ])
    vqa_dataset = VQADataset(input_dir, f'{data_type}.npy', max_qu_len=MAX_QU_LEN, transform= transform)
    dataloader = DataLoader(vqa_dataset, batch_size=batch_size, shuffle=False, num_workers=num_worker)

    qu_vocab_size = vqa_dataset.qu_vocab.vocab_size
    ans_vocab_size = vqa_dataset.ans_vocab.vocab_size

    model = VQAModel(feature_size=FEATURE_SIZE, qu_vocab_size=qu_vocab_size, ans_vocab_size=ans_vocab_size,
                     word_embed=WORD_EMBED, hidden_size=HIDDEN_SIZE, num_hidden=NUM_HIDDEN).to(device)
    model.load_state_dict(torch.load(ckpt_dir))
    model.eval()
    results = []

    for idx, sample in enumerate(dataloader):

        image = sample['image'].to(device)
        question = sample['question'].to(device)
        question_id = sample['question_id'].tolist()

        with torch.no_grad():
            logits = model(image, question)
            predict = torch.log_softmax(logits, dim=1)

        predict = torch.argmax(predict, dim=1).tolist()
        predict = [vqa_dataset.ans_vocab.idx2word(idx) for idx in predict]
        ans_qu_pair = [{'answer': ans, 'question_id': id} for ans, id in zip(predict, question_id)]
        results.extend(ans_qu_pair)
        if (idx+1) % 50 == 0:
            print(f'finishing {data_type} set : {(idx+1)*batch_size} / {len(vqa_dataset)}')

    if not os.path.exists(res_dir): os.makedirs(res_dir)
    with open(os.path.join(res_dir, f'v2_OpenEnded_mscoco_{data_type}2014_results.json'), 'w') as f:
        f.write(json.dumps(results))

if __name__ == "__main__":

    test(data_dir, 'val', batch_size=BATCH_SIZE, num_worker=8)